In [1]:
import time
from datetime import datetime
import random
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models

from sklearn.metrics import f1_score

from tqdm import tqdm
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'IMG_SIZE': 384,
    'EPOCHS': 60,
    'LEARNING_RATE': 0,
    'BATCH_SIZE': 32,
    'SEED': 41
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
import multiprocessing

num_workers = multiprocessing.cpu_count() // 2

In [6]:
df = pd.read_csv('./train.csv')

In [7]:
# https://dacon.io/competitions/official/236006/talkboard/407044?page=1&dtype=recent

df.loc[3896, 'artist'] = 'Titian'
df.loc[3986, 'id'] = 3986
df.loc[3986, 'artist'] = 'Alfred Sisley'
df.loc[3986, 'img_path'] = './train/3986.jpg'

In [8]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [9]:
train_df = df.sort_values(by=['id'])

In [10]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].values
    return df['img_path'].values, df['artist'].values

In [11]:
train_img_paths, train_labels = get_data(train_df)

In [12]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_paths)

- Cutout의 max_h_size, max_w_size가 32, 32였을 때와 64, 64 였을때 큰 차이는 없었으나  
    여기서는 max_h_size, max_w_size를 64, 64로 설정

In [13]:
train_transform = A.Compose([
                            A.Resize(800, 800),
                            A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),  # 약 4분의 1로 crop
                            A.HorizontalFlip(p=0.5),
                            A.Cutout(num_holes=4, max_h_size=64, max_w_size=64, p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [14]:
train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], 
                          shuffle=True, num_workers=num_workers)

- backbone 밑에 classifier layer를 추가하는 것과   
    backbone의 classifier layer를 초기화된 classifier layer로 교체하는 것 사이에  
    유의미한 성능 변화는 없어보였지만 여기서는 후자를 사용
    

In [15]:
import math
from Cream.TinyViT.models.tiny_vit import tiny_vit_21m_384

class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        
        self.backbone = tiny_vit_21m_384(pretrained=True)
        
        classifier = nn.Linear(576, num_classes)

        torch.nn.init.xavier_uniform_(classifier.weight)
        stdv = 1. / math.sqrt(classifier.weight.size(1))
        classifier.bias.data.uniform_(-stdv, stdv)
       
        self.backbone.head = classifier
          
    def forward(self, x):
        x = self.backbone(x)
        return x

In [16]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

- validation 없이 모두 학습에 사용

    train/valid 8:2로 학습했을 때 58 epoch에서 best score가 나왔기 때문에  
    동일한 조건으로 60 epoch 까지 학습

In [17]:
def train_without_validation(model, optimizer, criterion, train_loader, scheduler, device):
    
    model.to(device)
    
    for epoch in range(1, CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.to(device)
            
            optimizer.zero_grad()

            model_pred = model(img)
            
            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
        
        curr_lr = get_lr(optimizer)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] lr : [{curr_lr:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
                
    return model

In [18]:
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [19]:
model = BaseModel()
model.eval()

optimizer = torch.optim.Adam(params = model.parameters(), lr=CFG['LEARNING_RATE'])
criterion = nn.CrossEntropyLoss().to(device)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=10, T_mult=1, 
                                          eta_max=0.001,  T_up=3, gamma=0.5)

model = train_without_validation(model, optimizer, criterion, train_loader, scheduler, device)

LR SCALES: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


100%|██████████| 185/185 [01:47<00:00,  1.72it/s]

Epoch [1], Train Loss : [3.95779] lr : [0.00000]



100%|██████████| 185/185 [01:45<00:00,  1.75it/s]

Epoch [2], Train Loss : [2.07547] lr : [0.00033]


...

Epoch [58], Train Loss : [0.01026] lr : [0.00001]



100%|██████████| 185/185 [01:45<00:00,  1.75it/s]

Epoch [59], Train Loss : [0.01297] lr : [0.00001]



100%|██████████| 185/185 [01:45<00:00,  1.75it/s]

Epoch [60], Train Loss : [0.01126] lr : [0.00000]


## Inference

In [20]:
test_df = pd.read_csv('./test.csv')
test_img_paths = get_data(test_df, infer=True)
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=num_workers)

In [21]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    pred_probas = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            model_pred = model(img)
            
            pred_probas.extend(model_pred.detach().cpu().numpy().tolist())
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds, pred_probas

In [23]:
os.makedirs("./submit", exist_ok=True)

preds, logits = inference(model, test_loader, device)
preds = le.inverse_transform(preds)
submit = pd.read_csv('./sample_submission.csv')

# 제출 파일 생성
submit['artist'] = preds
submit.to_csv(f'./submit/novalid_answer.csv', index=False)

# ensemble을 위한 logit 파일 생성
submit["artist"] = logits
submit.to_csv(f"./submit/novalid_logit.csv", index=False)

100%|██████████| 396/396 [01:15<00:00,  5.27it/s]


Done.
